In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# import data
data_num = 9

raw_data = []

for i in range(data_num):
  raw_data.append(np.loadtxt('data/saxs-202411/S_LK-' + str(i + 1) + '.dat', skiprows=4, max_rows=1150).T)

raw_data = np.array(raw_data)

print(raw_data.shape)

labels = [
  '1 gL 0% HCl',
  '1 gL 40% HCl',
  '1 gL 60% HCl',
  '1 gL 80% HCl',
  '1 gL 100% HCl',
  '5 gL 40% HCl',
  '5 gL 60% HCl',
  '5 gL 80% HCl',
  '5 gL 100% HCl',
]

class_one = [0, 1, 2, 3, 4]

class_two = [5, 6, 7, 8]

class_three = [[1, 5], [2, 6], [3, 7], [4, 8]]

# set numpy print format
np.set_printoptions(precision=2, suppress=True)

In [ ]:
# config for plot
x_min = 0.008
x_max = 0.3
y_min = 0.8


def get_data_in_range(data, plot_title, req=False, x_min_=x_min, x_max_=x_max):
  q_raw = data[0]
  q = q_raw[(q_raw > x_min_) & (q_raw < x_max_)]
  Iq_raw = data[1]
  Iq = Iq_raw[(q_raw > x_min_) & (q_raw < x_max_)]

  lg_q = np.log(q)
  lg_Iq = np.log(Iq)

  if req:
    return lg_q, lg_Iq

  plt.figure()
  plt.plot(lg_q, lg_Iq)
  plt.xlabel('log(q)')
  plt.ylabel('log(I)')
  plt.title(plot_title)
  plt.show()

  return

In [ ]:
# plot zeroth data
plt.figure()
plt.loglog(raw_data[0, 0], raw_data[0, 1], label=labels[0])
plt.xlabel('q (1/A)')
plt.ylabel('Intensity (a.u.)')
plt.title('Zeroth data')
plt.xlim(x_min, x_max)
plt.ylim(y_min)
plt.legend()
plt.show()

In [ ]:
# plot all data
plt.figure()
for i in range(data_num):
  plt.loglog(raw_data[i, 0], raw_data[i, 1], label=labels[i])

plt.xlabel('q (1/A)')
plt.ylabel('Intensity (a.u.)')
plt.title('All data')
plt.xlim(x_min, x_max)
plt.ylim(y_min)
plt.legend()
plt.show()

In [ ]:
# plot data for calss one
plt.figure()
for i in class_one:
  plt.loglog(raw_data[i, 0], raw_data[i, 1], label=labels[i])

plt.xlabel('q (1/A)')
plt.ylabel('Intensity (a.u.)')
plt.title('Class One')
plt.xlim(x_min, x_max)
plt.ylim(y_min)
plt.legend()
plt.show()

In [ ]:
# plot data for calss two
plt.figure()
for i in class_two:
  plt.loglog(raw_data[i, 0], raw_data[i, 1], label=labels[i])

plt.xlabel('q (1/A)')
plt.ylabel('Intensity (a.u.)')
plt.title('Class Two')
plt.xlim(x_min, x_max)
plt.ylim(y_min)
plt.legend()
plt.show()

In [ ]:
# plt data for class three with subplots
plt.figure(figsize=(10, 10))
fig, axs = plt.subplots(2, 2)
for i, ax in enumerate(axs.flat):
  for j in class_three[i]:
    ax.loglog(raw_data[j, 0], raw_data[j, 1], label=labels[j])

  ax.set_xlabel('q (1/A)')
  ax.set_ylabel('Intensity (a.u.)')
  ax.set_title('Class Three ' + str(i + 1))
  ax.set_xlim(x_min, x_max)
  ax.set_ylim(y_min)
  ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
# for i in range(data_num):
# get_data_in_range(raw_data[i], plot_title=labels[i])

In [ ]:
def get_slopes(data, breakpoints, plot_title, req=False, show=True):
  q, Iq = get_data_in_range(data, 'data', req=True)

  if show:
    plt.plot(q, Iq)
    plt.xlabel('log(q)')
    plt.ylabel('log(I)')
    plt.title(plot_title)

  slopes = []  # store slopes
  flag = True  # flag for length of breakpoints if less than 2

  # check the length of breakpoints
  if len(breakpoints) < 2:
    print('breakpoints should have at least two elements')
    flag = False

  def label_template(T):
    return f'slope = {T:.2f}'

  # check the first part
  q_ = q[q < breakpoints[0]]
  Iq_ = Iq[q < breakpoints[0]]
  _ = np.polyfit(q_, Iq_, 1)
  p = np.poly1d(_)
  slopes.append(_[0])
  if show:
    plt.plot(q_, p(q_), '-', label=label_template(slopes[0]))

  # check the last part
  q_last = q[q > breakpoints[-1]]
  Iq_last = Iq[q > breakpoints[-1]]
  _last = np.polyfit(q_last, Iq_last, 1)
  p_last = np.poly1d(_last)
  last_slope = _last[0]  # store the last and temporary remove it from the list

  if flag:
    for i in range(len(breakpoints) - 1):
      q_ = q[(q > breakpoints[i]) & (q < breakpoints[i + 1])]
      Iq_ = Iq[(q > breakpoints[i]) & (q < breakpoints[i + 1])]
      _ = np.polyfit(q_, Iq_, 1)
      p = np.poly1d(_)
      slopes.append(_[0])
      if show:
        plt.plot(q_, p(q_), '-', label=label_template(slopes[-1]))

  # plot the last part
  slopes.append(last_slope)

  if show:
    plt.plot(q_last, p_last(q_last), '-', label=label_template(last_slope))

    plt.legend()
    plt.show()

  if req:
    return slopes

  return


# get_slopes(raw_data[0], [-4.4, -3.5, -2.6, -1.9, -1.4], plot_title=labels[0])
# get_slopes(raw_data[1], [-4.4, -3.5, -2.6, -1.9, -1.4], plot_title=labels[1])
# get_slopes(raw_data[2], [-3.7, -2.6, -1.9, -1.4], plot_title=labels[2])
# get_slopes(raw_data[3], [-3.7, -2.7, -1.9, -1.4], plot_title=labels[3])
# get_slopes(raw_data[4], [-3.6, -2.7, -1.9], plot_title=labels[4])
# get_slopes(raw_data[5], [-4.4, -3.6, -1.5], plot_title=labels[5])
# get_slopes(raw_data[6], [-3.8, -2.5, -1.5], plot_title=labels[6])
# get_slopes(raw_data[7], [-3.5, -2.8, -1.5], plot_title=labels[7])
# get_slopes(raw_data[8], [-3.3, -2.5, -1.5], plot_title=labels[8])

all_breakpoints = [
  [-4.4, -3.5, -2.6, -1.9, -1.4],
  [-4.4, -3.5, -2.6, -1.9, -1.4],
  [-3.7, -2.6, -1.9, -1.4],
  [-3.7, -2.7, -1.9, -1.4],
  [-3.6, -2.7, -1.9],
  [-4.4, -3.6, -1.5],
  [-3.8, -2.5, -1.5],
  [-3.3, -2.8, -1.5],
  [-3.3, -2.5, -1.5],
]

slope_data = []  # store all slopes data

for i in range(data_num):
  slope = get_slopes(raw_data[i], all_breakpoints[i], plot_title=labels[i], req=True, show=False)
  print(f'slope for {labels[i]}: {np.array(slope)}')
  slope_data.append(slope)